In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
df=pd.read_csv('/content/samsum-train.csv')
df = df.dropna()
df.to_csv('/content/samsum-train.csv', index=False)

In [ ]:
import pandas as pd

# Paths to the individual CSV files
train_file = '/content/samsum-train.csv'  # Update these paths accordingly
test_file = '/content/samsum-test.csv'
validation_file = '/content/samsum-validation.csv'

# Load the data into pandas DataFrames
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
validation_data = pd.read_csv(validation_file)

# Check if all files have the same columns and structure
print(train_data.columns)
print(test_data.columns)
print(validation_data.columns)

# Merge all three files into a single DataFrame
merged_data = pd.concat([train_data, test_data, validation_data], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_file = '/content/merged_data.csv'
merged_data.to_csv(merged_file, index=False)

print(f"Data has been successfully merged and saved to {merged_file}.")


Index(['id', 'dialogue', 'summary'], dtype='object')
Index(['id', 'dialogue', 'summary'], dtype='object')
Index(['id', 'dialogue', 'summary'], dtype='object')
Data has been successfully merged and saved to /content/merged_data.csv.


In [ ]:
import pandas as pd

def remove_long_rows_from_csv(input_file, output_file, max_dialogue_tokens=400, max_summary_tokens=50):
    """
    Remove rows from a CSV file where the `dialogue` column has token count >= max_dialogue_tokens
    or the `summary` column has token count >= max_summary_tokens.

    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the filtered CSV file.
        max_dialogue_tokens (int): Maximum allowed tokens in `dialogue` column.
        max_summary_tokens (int): Maximum allowed tokens in `summary` column.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_file)

    # Ensure 'dialogue' and 'summary' columns exist
    if 'dialogue' not in df.columns or 'summary' not in df.columns:
        raise ValueError("Input file must contain 'dialogue' and 'summary' columns.")

    # Calculate token counts for `dialogue` and `summary`
    df['dialogue_token_count'] = df['dialogue'].apply(lambda x: len(str(x).split()))
    df['summary_token_count'] = df['summary'].apply(lambda x: len(str(x).split()))

    # Filter rows based on token count limits
    filtered_df = df[
        (df['dialogue_token_count'] < max_dialogue_tokens) &
        (df['summary_token_count'] < max_summary_tokens)
    ]

    # Drop the temporary token count columns
    filtered_df = filtered_df.drop(columns=['dialogue_token_count', 'summary_token_count'])

    # Save the filtered DataFrame back to a CSV file
    filtered_df.to_csv(output_file, index=False)

# Example usage:
input_csv_path = "/content/First_Row_New_Text_Document.csv"  # Replace with the path to your input CSV file
output_csv_path = "/content/First_Row_New_Text_Document.csv"  # Replace with the path to save the filtered CSV

remove_long_rows_from_csv(input_csv_path, output_csv_path, max_dialogue_tokens=400, max_summary_tokens=50)

print(f"Filtered data saved to {output_csv_path}")


Filtered data saved to /content/First_Row_New_Text_Document.csv


##Data Cleaning

In [ ]:
import pandas as pd
import re
df = pd.read_csv('/content/First_Row_New_Text_Document.csv')
#df = df.drop(columns=['id'])
df['dialogue'] = df['dialogue'].str.replace(r'\r\n', ' ', regex=True)
df['summary'] = df['summary'].str.replace(r'\r\n', ' ', regex=True)
def clean_text(text):
    # Remove everything except letters
    return re.sub(r'[^a-zA-Z\s]', '', text)
# Apply the cleaning function to the 'dialogue' column
df['dialogue'] = df['dialogue'].apply(clean_text)
df['summary'] = df['summary'].apply(clean_text)
# Save the cleaned dataset to a new CSV file
df = df.rename(columns={'dialogue': 'en', 'summary': 'it'})
df.to_csv('/content/First_Row_New_Text_Document.csv', index=False)
print(df.iloc[:, -3:])
print("The text has been cleaned and saved to 'cleaned.csv'")


             id                                                 en  \
0      13818513  Amanda I baked  cookies Do you want some Jerry...   
1      13728867  Olivia Who are you voting for in this election...   
2      13681000  Tim Hi whats up Kim Bad mood tbh I was going t...   
3      13730747  Edward Rachel I think Im in ove with Bella rac...   
4      13728094  Sam hey  overheard rick say something Sam i do...   
...         ...                                                ...   
15969  13611821  Chris Im on my way Peter ok great Chris are we...   
15970  13829423  Carla Ive got it Diego what Carla my date for ...   
15971  13829261  Julia Greg just texted me Robert ugh delete hi...   
15972  13680226  Marry I broke my nail  Tina oh no Marry u know...   
15973  13862383  Paige I asked them to wait and send the declar...   

                                                      it  
0      Amanda baked cookies and will bring Jerry some...  
1      Olivia and Olivier are voting for 

##Data Conversion


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load the dataset from the CSV file
file_path = "/content/First_Row_New_Text_Document.csv"

# Step 1: Load the dataset without forcing column names and using the correct delimiter
df = pd.read_csv(file_path, delimiter=",", header=0)

# Step 2: Check if the columns loaded correctly
print("Loaded DataFrame:")
print(df.head())

# Step 3: Add a unique 'id' column
df["id"] = df.index.astype(str)

# Step 4: Create a 'translation' column
df["translation"] = df[["en", "it"]].apply(lambda x: {"en": x["en"], "it": x["it"]}, axis=1)

# Step 5: Drop the original columns to keep only 'id' and 'translation'
df = df[["id", "translation"]]

# Step 6: Convert to a Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Step 7: Wrap it into a DatasetDict
dataset_dict = DatasetDict({"train": hf_dataset})

# Step 8: Save the dataset locally
output_path = "./converted_conversation_dataset"
dataset_dict.save_to_disk(output_path)

print(f"Dataset converted and saved successfully at: {output_path}")


Loaded DataFrame:
         id                                                 en  \
0  13818513  Amanda I baked  cookies Do you want some Jerry...   
1  13728867  Olivia Who are you voting for in this election...   
2  13681000  Tim Hi whats up Kim Bad mood tbh I was going t...   
3  13730747  Edward Rachel I think Im in ove with Bella rac...   
4  13728094  Sam hey  overheard rick say something Sam i do...   

                                                  it  
0  Amanda baked cookies and will bring Jerry some...  
1  Olivia and Olivier are voting for liberals in ...  
2  Kim may try the pomodoro technique recommended...  
3  Edward thinks he is in love with Bella Rachel ...  
4  Sam is confused because he overheard Rick comp...  


Saving the dataset (0/1 shards):   0%|          | 0/15974 [00:00<?, ? examples/s]

Dataset converted and saved successfully at: ./converted_conversation_dataset


In [ ]:
from datasets import load_from_disk

# Load the saved dataset
dataset = load_from_disk("/content/converted_conversation_dataset")

# Inspect the structure
print(dataset)
print(dataset["train"][0])  # First record


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 15974
    })
})
{'id': '0', 'translation': {'en': 'Amanda I baked  cookies Do you want some Jerry Sure Amanda Ill bring you tomorrow ', 'it': 'Amanda baked cookies and will bring Jerry some tomorrow'}}


## Embeding Scaling and Forward


In [ ]:
import torch
import torch.nn as nn
import math

class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

## Positional Embeding


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

## Layer Normalization

In [ ]:
class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

## Feed Forward

In [ ]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

## Multi Head Attentation

In [ ]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        return self.w_o(x)


## Residual Connection

In [ ]:
class ResidualConnection(nn.Module):

        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)

        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

## Encoder

In [ ]:
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

In [ ]:
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

## Decoder

In [ ]:
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

In [ ]:
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

## Projection Layer

In [ ]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)

## Transformer

In [ ]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, d_model)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)

In [ ]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

## Greedy Decode

In [ ]:
import torch
import torch.nn as nn

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
from torch.utils.data import Dataset,DataLoader,random_split

In [ ]:
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source, source_mask)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
    while True:
        if decoder_input.size(1) == max_len:
            break

        # build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # get next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)



In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 46.8 MB/s eta 0:00:00


## Run Validation

In [ ]:
import torchmetrics
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        # get the console window width
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        # If we can't get the console width, use 80 as default
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device) # (b, seq_len)
            encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)

            # check that the batch size is 1
            assert encoder_input.size(
                0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            # Print the source, target and model output
            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break

    if writer:
        # Evaluate the character error rate
        # Compute the char error rate
        metric = torchmetrics.CharErrorRate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation cer', cer, global_step)
        writer.flush()

        # Compute the word error rate
        metric = torchmetrics.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation wer', wer, global_step)
        writer.flush()

        # Compute the BLEU metric
        metric = torchmetrics.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation BLEU', bleu, global_step)
        writer.flush()

## Tokenzier

In [ ]:
def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

In [ ]:
def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

In [ ]:
from datasets import load_from_disk
def get_ds(config):
    # It only has the train split, so we divide it overselves
    dataset_path = f"{config['datasource']}"
    ds_raw = load_from_disk(dataset_path)
    ds_raw = ds_raw['train']

    # Build tokenizers
    tokenizer_src = get_or_build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config['lang_tgt'])

    # Keep 90% for training, 10% for validation
    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    # Find the maximum length of each sentence in the source and target sentence
    max_len_src = 0
    max_len_tgt = 0

    for item in ds_raw:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f'Max length of source sentence: {max_len_src}')
    print(f'Max length of target sentence: {max_len_tgt}')


    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)
    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

## Get Model

In [ ]:
def get_model(config, vocab_src_len, vocab_tgt_len):
    model = build_transformer(vocab_src_len, vocab_tgt_len, config["seq_len"], config['seq_len'], d_model=config['d_model'])
    return model

## Train Model

In [ ]:
from pathlib import Path
def train_model(config):
    # Define the device
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps or torch.backends.mps.is_available() else "cpu"
    print("Using device:", device)
    if (device == 'cuda'):
        print(f"Device name: {torch.cuda.get_device_name(device.index)}")
        print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
    elif (device == 'mps'):
        print(f"Device name: <mps>")
    else:
        print("NOTE: If you have a GPU, consider using it for training.")
        print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
        print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
    device = torch.device(device)

    # Make sure the weights folder exists
    Path(f"{config['datasource']}_{config['model_folder']}").mkdir(parents=True, exist_ok=True)

    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
    # Tensorboard
    writer = SummaryWriter(config['experiment_name'])

    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

    # If the user specified a model to preload before training, load it
    initial_epoch = 0
    global_step = 0
    preload = config['preload']
    model_filename = latest_weights_file_path(config) if preload == 'latest' else get_weights_file_path(config, preload) if preload else None
    if model_filename:
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
    else:
        print('No model to preload, starting from scratch')

    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    for epoch in range(initial_epoch, config['num_epochs']):
        torch.cuda.empty_cache()
        model.train()
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        for batch in batch_iterator:

            encoder_input = batch['encoder_input'].to(device) # (b, seq_len)
            decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
            encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
            decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)

            # Run the tensors through the encoder, decoder and the projection layer
            encoder_output = model.encode(encoder_input, encoder_mask) # (B, seq_len, d_model)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) # (B, seq_len, d_model)
            proj_output = model.project(decoder_output) # (B, seq_len, vocab_size)

            # Compare the output with the label
            label = batch['label'].to(device) # (B, seq_len)

            # Compute the loss using a simple cross entropy
            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            # Log the loss
            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            # Backpropagate the loss
            loss.backward()

            # Update the weights
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

            global_step += 1

        # Run validation at the end of every epoch
        run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)

        # Save the model at the end of every epoch
        model_filename = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)


## Running the model

In [ ]:
import warnings
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    config = get_config()
    train_model(config)

Using device: cuda
Device name: Tesla T4
Device memory: 14.74810791015625 GB
Max length of source sentence: 399
Max length of target sentence: 49
No model to preload, starting from scratch


Processing Epoch 00: 100%|██████████| 1797/1797 [16:54<00:00,  1.77it/s, loss=5.854]


--------------------------------------------------------------------------------
    SOURCE: Terry Where are you Gordon Breakfast at Mats Terry good Im nearby Ill come over to give you back the money
    TARGET: Gordon is eating breakfast at mats Terry is going to meet him there to give him the money back
 PREDICTED: is going to the new new new new place
--------------------------------------------------------------------------------
    SOURCE: Ron I too have plans for the weekend Thanks for asking btw Taylor I was going to So what are ur plans Harry Oh did the princess get offended P Ron Harry not as much as yo mama Ron Taylor Im going to an art exhibition  Harry You bastard Thats it Art exhibition Taylor Yeah whats the fun in that Ron Thanks  ur support Actually my pictures are going to be there too Im one of the authors Taylor Ur first exhibition Congratulations Im so happy  u Harry Y didnt u say I wouldve cancelled or put off my plans Ron Nah thats fine Thanks guys
    TARGET: At 

Processing Epoch 01: 100%|██████████| 1797/1797 [16:55<00:00,  1.77it/s, loss=5.786]


--------------------------------------------------------------------------------
    SOURCE: Elie  So youre in NY now welcome  Bob yep its great to be here at last Elie How is your hotel Bob its fine my room is nice Elie Do you have a nice view from your window Bob i can see plenty of buildings Elie ill come and pick you up at am  Ill want you to  meet my team and Barbara Do you remember her  Bob Uhh sorry Elie No im the one who  is sorry You must be tired Bob yes im  a bit Elie True its  am in London Bob yes i need to go to bed Im too tired to go down and have diner Elie youre right its time to go  to bed Ill wait for you tomorrow at your hotel Bob Perfect thanks Elie just one thing Bob Whats that Elie Don t be late lol Will you be ok in the tube Bob ill take a cab Elie ok but in this case youd better leave earlier Bob Right i forgot about this tremendous traffic in NY city Elie Fine its gonna be great to see you again Bob yes im happy to see you too tomorrow Elie have a good night Bo

...

--------------------------------------------------------------------------------
    SOURCE: Fabian guys were seeing each other on the match tomorrow am I right Max yep Albert sure Fabian I suggest going to some pub to watch it what do you think at some places theyve got nice and huge TVs and we can also eat and drink Max wanna know whos got a huge TV ME so we can meet at my place and drink liters of beer and eat tones of food and spend less money Max and have more privacy Albert sounds really cool if that wont be a problem then Id come Fabian hmmm actually youre rightso what time Max what time does it start Albert pm Max hmmmmm come to my place at  well make a before party and prepare some food even a dinner ribs or whatever Albert cooooooooool Fabian ok I agree I can buy the meat Albert and I can buy alcohol at least the first round hahahah Max cool settled then
    TARGET: Tomorrow Fabian Max and Albert are going to watch a match at Maxs place Theyre going to meet at  and prepare fo

Processing Epoch 05: 100%|██████████| 1797/1797 [16:55<00:00,  1.77it/s, loss=4.852]


--------------------------------------------------------------------------------
    SOURCE: Erica Guys we want to sell our car finally if you know anybody who could be interested let me know please Tom How old is it Its Renault laguna right Erica yes its  Barry its really old Heather but good for youngsters I think Erica I agree because its very safe and reliable thats why weve used it so long Erica there is only one disadvantage  it burns a lot of gas Tom sure its normal like all old cars
    TARGET: Erica wants to sell her Renault laguna from 
 PREDICTED: Dan is going to visit his car He is looking for a car
--------------------------------------------------------------------------------
    SOURCE: Gregory fileother tickets now Sue in Chuck me too Gregory Ill buy all  so were next to each other Sue sure Ill pay you back in a minute Chuck filegif Gregory I got them Sue D youre t he best Chuck Awesome dude Gregory Just barely Theyre already gone Gregory Now you owe me a  bucks each P

Processing Epoch 06:  57%|█████▋    | 1018/1797 [09:35<07:20,  1.77it/s, loss=4.435]


KeyboardInterrupt: 

## Data set configration

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class BilingualDataset(Dataset):

    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()
        self.seq_len = seq_len

        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        # Transform the text into tokens
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        # Add sos, eos and padding to each sentence
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2  # We will add <s> and </s>
        # We will only add <s>, and </s> only on the label
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1

        # Make sure the number of padding tokens is not negative. If it is, the sentence is too long
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        # Add <s> and </s> token
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only <s> token
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only </s> token
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Double check the size of the tensors to make sure they are all seq_len long
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,  # (seq_len)
            "decoder_input": decoder_input,  # (seq_len)
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len),
            "label": label,  # (seq_len)
            "src_text": src_text,
            "tgt_text": tgt_text,
        }

In [ ]:
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = latest_weights_file_path(config)
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Using device: cuda
Max length of source sentence: 366
Max length of target sentence: 49


<All keys matched successfully>

In [ ]:
run_validation(
    model,
    val_dataloader,
    tokenizer_src,
    tokenizer_tgt,
    config['seq_len'],
    device,
    lambda msg: print(msg),
    0,
    None,
    num_examples=1
)


--------------------------------------------------------------------------------
    SOURCE: Sofia I think my car needs an oil change baby David Do you remember when was the last time you changed it Sofia I think it was when my car had like k miles on it David And how many it has now Sofia k David Then yes you definitely need an oil change Sofia Should I do it at the dealer or a regular body shop David If you have a coupon you can make an appointment with at the dealer If you dont have it itll be more  Sofia I do have a  coupon sent in the mail David Then go ahead and schedule an appointment maybe for Saturday that way I can take it if you want Sofia Perfect Ill give them a call and check Saturday David Sounds good
    TARGET: Sofias car needs an oil change Sofia will make an appointment for Saturday so David can take her car
 PREDICTED: car needs an change will make an appointment for Saturday so David can take her car
------------------------------------------------------------------

In [ ]:
t = Summery("Rose hey at what time tomorrow Rose filephoto Anna the screening at pm looks good Zara nooo thats too late Anna you need to be back home before midnight XD Zara  Zara im seeing Jack at pm Anna Jack  ok youre excused Zara pm Anna fine for me i guess Zara Rose Rose sure pm is perfect for me")
print(t)

Using device: cuda
    SOURCE: Rose hey at what time tomorrow Rose filephoto Anna the screening at pm looks good Zara nooo thats too late Anna you need to be back home before midnight XD Zara  Zara im seeing Jack at pm Anna Jack  ok youre excused Zara pm Anna fine for me i guess Zara Rose Rose sure pm is perfect for me
 PREDICTED: They are at the  at the  is going to see Jack at pm  They are at the at the is going to see Jack at pm


## Config

In [ ]:
def get_config():
    return {
        "batch_size": 8,
        "num_epochs": 200,
        "lr": 10**-4,
        "seq_len": 500,
        "d_model": 512,
        "datasource": '/content/converted_conversation_dataset',  # Local path
        "lang_src": "en",
        "lang_tgt": "it",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

In [ ]:
def get_weights_file_path(config, epoch: str):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

# Find the latest weights file in the weights folder
def latest_weights_file_path(config):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}*"
    weights_files = list(Path(model_folder).glob(model_filename))
    if len(weights_files) == 0:
        return None
    weights_files.sort()
    return str(weights_files[-1])

In [ ]:
from pathlib import Path
from tokenizers import Tokenizer
from datasets import load_dataset
import torch
import sys
from datasets import load_from_disk
def Summery(sentence: str):
    # Define the device, tokenizers, and model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    config = get_config()
    tokenizer_src = Tokenizer.from_file(str(Path(config['tokenizer_file'].format(config['lang_src']))))
    tokenizer_tgt = Tokenizer.from_file(str(Path(config['tokenizer_file'].format(config['lang_tgt']))))
    model = build_transformer(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size(), config["seq_len"], config['seq_len'], d_model=config['d_model']).to(device)

    # Load the pretrained weights
    model_filename = latest_weights_file_path(config)
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])

    # if the sentence is a number use it as an index to the test set
    label = ""
    if type(sentence) == int or sentence.isdigit():
        id = int(sentence)
        dataset_path = f"{config['datasource']}"
        ds_raw = load_from_disk(dataset_path)
        ds = ds_raw['train']
        ds = BilingualDataset(ds, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
        sentence = ds[id]['src_text']
        label = ds[id]["tgt_text"]
    seq_len = config['seq_len']

    # translate the sentence
    model.eval()
    with torch.no_grad():
        # Precompute the encoder output and reuse it for every generation step
        source = tokenizer_src.encode(sentence)
        source = torch.cat([
            torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64),
            torch.tensor(source.ids, dtype=torch.int64),
            torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64),
            torch.tensor([tokenizer_src.token_to_id('[PAD]')] * (seq_len - len(source.ids) - 2), dtype=torch.int64)
        ], dim=0).to(device)
        source_mask = (source != tokenizer_src.token_to_id('[PAD]')).unsqueeze(0).unsqueeze(0).int().to(device)
        encoder_output = model.encode(source, source_mask)

        # Initialize the decoder input with the sos token
        decoder_input = torch.empty(1, 1).fill_(tokenizer_tgt.token_to_id('[SOS]')).type_as(source).to(device)

        # Print the source sentence and target start prompt
        if label != "": print(f"{f'ID: ':>12}{id}")
        print(f"{f'SOURCE: ':>12}{sentence}")
        if label != "": print(f"{f'TARGET: ':>12}{label}")
        print(f"{f'PREDICTED: ':>12}", end='')

        # Generate the translation word by word
        while decoder_input.size(1) < seq_len:
            # build mask for target and calculate output
            decoder_mask = torch.triu(torch.ones((1, decoder_input.size(1), decoder_input.size(1))), diagonal=1).type(torch.int).type_as(source_mask).to(device)
            out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

            # project next token
            prob = model.project(out[:, -1])
            _, next_word = torch.max(prob, dim=1)
            decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1)

            # print the translated word
            print(f"{tokenizer_tgt.decode([next_word.item()])}", end=' ')

            # break if we predict the end of sentence token
            if next_word == tokenizer_tgt.token_to_id('[EOS]'):
                break

    # convert ids to tokens
    return tokenizer_tgt.decode(decoder_input[0].tolist())

#read sentence from argument
#translate(sys.argv[1] if len(sys.argv) > 1 else "I am not a very good a student.")